In [1]:
import requests
import pprint
import urllib3
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas.io.json import json_normalize
import numpy as np
import matplotlib
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel

output_notebook()

Loading BokehJS ...

In [2]:
#get access token (good for 6 hours max)
def get_token():
    auth_url = 'https://www.strava.com/oauth/token' 

    data = {
        'client_id' : '48600',
        'client_secret': '9cf6f6d5e2758063ec1cd70d10878dcf57abf5cb',
        'refresh_token': 'b353ffb23a0f9b4ccecb9f13f5dde52d45262cf6',
        'grant_type': 'refresh_token',
        'f': 'json',
    }

    r = requests.post(auth_url, data=data, verify=False)
    access_token = r.json()['access_token']
    token_expiry = r.json()['expires_at']
    return access_token, token_expiry

#get activity stats to obtain total number of rides
def get_alltime_activities(access_token):
    #col_names = ['id', 'type']
   # activities_df = pd.DataFrame(columns = col_names)
    url = "https://www.strava.com/api/v3/activities"

    activity_headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {access_token}',
    }

    page = 1
    alltime_activities_list = []
    while True:
        activity_params = (
        ('page', page),
        ('per_page', '200'),
        )

        # get page of activities from Strava
        a_r = requests.get(url, headers=activity_headers, params=activity_params)
        a_r = a_r.json()

        # if no results then exit loop
        if (not a_r):
            break
            
        # otherwise add new data to dataframe
        for x in range(len(a_r)):
            alltime_activities_list.append(a_r[x])
            #activities_df.loc[x + (page-1)*50,'id'] = a_r[x]['id']
            #activities_df.loc[x + (page-1)*50,'type'] = a_r[x]['type']
        
        # increment page
        page += 1
        
        return json_normalize(alltime_activities_list)
    
#get activity details for up to 100 activities (API calls limited)
def get_activity_details(qry_df, access_token):
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {access_token}',
    }
    params = (
        ('include_all_efforts', 'true'),
    )
    all_rides = []
    for x in range(len(qry_df)):
        #url = f'https://www.strava.com/api/v3/activities/{activities_qry_df.iloc[x,0]}'
        url = f'https://www.strava.com/api/v3/activities/{qry_df.iloc[x]}'
        rides_r = requests.get(url, headers=headers, params=params)
        rides_r = rides_r.json()
        all_rides.append(rides_r)
    all_rides_df = json_normalize(all_rides)
    
    return all_rides_df

In [3]:
access_token_info = get_token() #[token, exp time]

#json_normalize outputs a df
all_activities_summary_df = get_alltime_activities(access_token_info[0])

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [19]:
activity_type = 'Ride'

#get list of all Ride/Virtual Ride activities
activities_qry_df = all_activities_summary_df[all_activities_summary_df['type'].str.contains(f'{activity_type}')]

#get details incl segments of every activity (1 API call per activities)
activities_detail_df = get_activity_details(activities_qry_df['id'], access_token_info[0])

In [22]:
activities_detail_df.to_csv('alltime_activities.csv')

In [20]:
#get list of activity IDs that still need to be queried
temp_df = activities_qry_df['id'].append(activities_detail_df['id'], ignore_index=True)
temp_df.dropna()
temp_df.drop_duplicates(keep=False, inplace=True)

In [21]:
#activities_detail_df
mylist = []

#drop empty rows
activities_detail_df.dropna(subset=['id'], inplace=True)

for x in range(len(activities_detail_df)):
    data = json_normalize(activities_detail_df['segment_efforts'].iloc[x])
    mylist.append(data)
    
segments_df = pd.concat(mylist, ignore_index=True, sort = True)
segments_df.to_csv('alltime_segments.csv',index=False)

In [4]:
segments_df = pd.read_csv('alltime_segments.csv')
activities_df = pd.read_csv('alltime_activities.csv')

In [6]:
segments_df.head()
activities_df.head()

,Unnamed: 0,achievement_count,athlete.id,athlete.resource_state,athlete_count,available_zones,average_cadence,average_heartrate,average_speed,average_watts,calories,comment_count,commute,description,device_name,device_watts,display_hide_heartrate_option,distance,elapsed_time,elev_high,elev_low,embed_token,end_latlng,external_id,flagged,from_accepted_tag,gear.distance,gear.id,gear.name,gear.primary,gear.resource_state,gear_id,has_heartrate,has_kudoed,heartrate_opt_out,id,kilojoules,kudos_count,laps,location_city,location_country,location_state,manual,map.id,map.polyline,map.resource_state,map.summary_polyline,max_heartrate,max_speed,max_watts,moving_time,name,perceived_exertion,photo_count,photos.count,photos.primary,photos.primary.id,photos.primary.source,photos.primary.unique_id,photos.primary.urls.100,photos.primary.urls.600,photos.use_primary_photo,pr_count,prefer_perceived_exertion,private,resource_state,segment_efforts,splits_metric,splits_standard,start_date,start_date_local,start_latitude,start_latlng,start_longitude,timezone,total_elevation_gain,total_photo_count,trainer,type,upload_id,upload_id_str,utc_offset,visibility,weighted_average_watts,workout_type
0,0,0,14175897,1,2,[],NaN,NaN,5.199,119.8,2214.8,2,False,NaN,Strava iPhone App,False,False,86219.6,23790,459.4,331.4,18749a4ca03725f45787579a60f064f9b7234634,"[43.570892, -80.507684]",9A13F6D0-D12F-4A63-815A-1480D3C09EB4-activity.fit,False,False,2105733.0,b5707375,Red Gravel Bike,False,2.0,b5707375,False,False,False,3574471561,1986.4,4,"[{'id': 11823262427, 'resource_state': 2, 'nam...",NaN,NaN,NaN,False,a3574471561,_gihGdlwjNBHDDB?NEr@_AROD?NFRRh@v@dBlBz@lA^fAP...,3,sghhGr_xjN|Oln@wBiD?b@}Al@oHbOcBpHwClDy@~C_DlD...,NaN,16.0,NaN,16585,century except strava paused for 15 km 😡,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False,3,"[{'id': 2704460581770084427, 'resource_state':...","[{'distance': 1000.4, 'elapsed_time': 173, 'el...","[{'distance': 1611.1, 'elapsed_time': 319, 'el...",2020-06-06T14:38:59Z,2020-06-06T10:38:59Z,43.470088,"[43.470088, -80.570422]",-80.570422,(GMT-05:00) America/Toronto,652.4,0,False,Ride,3.816336e+09,3.816336e+09,-14400.0,followers_only,NaN,10.0
1,1,5,14175897,1,2,[],NaN,NaN,2.850,120.6,877.8,0,False,NaN,Strava iPhone App,False,False,18596.6,6742,409.9,359.5,f9d86c99542b68a4d67d5b8db1ff2d567d417d12,"[43.433133, -80.560314]",8087F948-6EC4-401B-B1CC-D6C61EC9E739-activity.fit,False,False,91120.0,b7150813,Black FS,True,2.0,b7150813,False,False,False,3565252972,787.3,1,"[{'id': 11794450219, 'resource_state': 2, 'nam...",NaN,NaN,NaN,False,a3565252972,g`bhGvlujNJMB??CCDC??BBUVc@Xs@b@_@|@m@tAyAFAJB...,3,g`bhGvlujN|@uBnE_Dr@bDnJxQpC~Ab@`C`B~CdBT@pAn@...,NaN,8.8,NaN,6526,Afternoon Ride,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,False,3,"[{'id': 2703764858871674877, 'resource_state':...","[{'distance': 1000.7, 'elapsed_time': 255, 'el...","[{'distance': 1609.9, 'elapsed_time': 389, 'el...",2020-06-04T21:23:53Z,2020-06-04T17:23:53Z,43.433163,"[43.433163, -80.560275]",-80.560275,(GMT-05:00) America/Toronto,264.7,0,False,Ride,3.806592e+09,3.806592e+09,-14400.0,followers_only,NaN,10.0
2,2,1,14175897,1,1,[],NaN,NaN,5.222,106.5,742.6,0,False,NaN,Strava iPhone App,False,False,32660.6,6374,373.9,322.5,eabfbaca581cb6d9f4982e709c15ea1f274b2a40,"[43.470145, -80.570649]",6D945900-2D28-46F9-86B6-748EBAF34A7D-activity.fit,False,False,2105733.0,b5707375,Red Gravel Bike,False,2.0,b5707375,False,False,False,3555380594,666.0,1,"[{'id': 11765186359, 'resource_state': 2, 'nam...",NaN,NaN,NaN,False,a3555380594,yfihGtlwjNA@D@VEtAg@ILa@PWNI?S[A@EC?FMZUb@a@^M...,3,gcihGloxjN\j@RlAg@p@GVh@fDApAa@bG\f@r@jBNz@FtE...,NaN,13.7,NaN,6254,Afternoon Ride,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,False,3,"[{'id': 2703039393098004829, 'resource_state':...","[{'distance': 1000.7, 'elapsed_time': 268, 'el...","[{'distance': 1611.9, 'elapsed_time': 403, 'el...",2020-06-02T21:27:02Z,2020-06-02T17:27:02Z,43.470059,"[43.470059, -80.570507]",-80.570507,(GMT-05:00) America/Toronto,203.5,0,False,R

In [7]:
segments_df['name'].unique()

array(['SJMA mini climb east side', 'Erbsville Rd Town line sprint',
       'Erbsville - SJAM to Bisch', ...,
       'Sick Downhill Trail with Ramp-Jump in Reverse',
       'This Camel Has Two Humps', 'bootleg. up & back to the start.'],
      dtype=object)

In [8]:
sample = segments_df[segments_df['name'].str.contains('Bisch Bash')]

In [10]:
#plot historical segment performance

date_format="%Y-%m-%d"
sample["start_date_local"] = pd.to_datetime(sample['start_date_local'], format=date_format)

p=figure(x_axis_type='datetime')
p.scatter(sample.loc[:,'start_date_local'], sample.loc[:,'elapsed_time'])

show(p)

print(sample.start_date_local)

#gonna use bokeh

# p.circle(x, y, size=10, color='red', legend='circle')
# p.line(x, y, color='blue', legend='line')
# p.triangle(y, x, color='gold', size=10, legend='triangle')

#show(p)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


4     2020-06-06 10:53:13
66    2020-06-02 17:43:00
128   2020-05-30 10:24:29
140   2020-05-26 17:36:43
190   2020-05-23 10:09:38
220   2020-05-21 17:26:30
239   2020-05-16 10:20:35
258   2020-05-13 17:47:29
324   2020-05-03 11:56:32
361   2020-05-01 17:54:15
380   2020-04-27 18:00:00
Name: start_date_local, dtype: datetime64[ns]


In [ ]:
import numpy  as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import bokeh
import pandas as pd
from datetime import datetime as dt

bokeh.sampledata.download()
from bokeh.sampledata.stocks import AAPL

output_notebook()

In [13]:
df = pd.DataFrame.from_dict(AAPL)

# Convert date column in df from strings to the proper datetime format
date_format="%Y-%m-%d"
df["date"] = pd.to_datetime(df['date'], format=date_format)
# Use the same conversion for selected dates
weekapple = df[(df.date>=dt.strptime("2000-03-01", date_format)) & 
               (df.date<=dt.strptime("2000-04-01", date_format))]

p = figure(x_axis_type="datetime", title="AAPL", plot_height=350, plot_width=800)
p.xgrid.grid_line_color=None
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Value'
p.scatter(weekapple.date, weekapple.close)
show(p)

print(weekapple)

         date    open    high     low   close    volume  adj_close
0  2000-03-01  118.56  132.06  118.50  130.31  38478000      31.68
1  2000-03-02  127.00  127.94  120.69  122.00  11136800      29.66
2  2000-03-03  124.87  128.23  120.00  128.00  11565200      31.12
3  2000-03-06  126.00  129.13  125.00  125.69   7520000      30.56
4  2000-03-07  126.44  127.44  121.12  122.87   9767600      29.87
5  2000-03-08  122.87  123.94  118.56  122.00   9690800      29.66
6  2000-03-09  120.87  125.00  118.25  122.25   9884400      29.72
7  2000-03-10  121.69  127.94  121.00  125.75   8900800      30.57
8  2000-03-13  122.12  126.50  119.50  121.31  10864400      29.50
9  2000-03-14  121.22  124.25  114.00  114.25  15321200      27.78
10 2000-03-15  115.62  120.25  114.12  116.25  15845200      28.26
11 2000-03-16  117.31  122.00  114.50  121.56  13516800      29.56
12 2000-03-17  120.12  125.00  119.62  125.00  10902400      30.39
13 2000-03-20  123.50  126.25  122.37  123.00   7316400      2

In [ ]:
print(df)